In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image 

In [2]:
#Đọc dữ liệu
data = pd.read_csv('A_Z Handwritten Data.csv')
data = data.astype('float32')

In [3]:
#Hiển thị 5 hàng dữ liệu đầu để kiểm tra
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#Tách dữ liệu và nhãn
X = data.drop('0', axis=1).values
y = data['0'].values

In [5]:
#Chuẩn hóa dữ liệu
X /= 255.0

In [6]:
#Chuyển đổi nhãn từ dạng số nguyên sang dạng one-hot encoding.
y = to_categorical(y, num_classes=26)

In [7]:
#Định hình lại dữ liệu đầu vào thành kích thước (số lượng mẫu, 28, 28, 1).
X = X.reshape(-1, 28, 28, 1)

In [8]:
#Chia dữ liệu thành tập huấn luyện và tập kiểm tra
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#Kiểm tra kích thước dữ liệu
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Training data shape: (297960, 28, 28, 1)
Test data shape: (74490, 28, 28, 1)


In [10]:
def build_cnn_model(input_shape, num_classes):
    model = Sequential()
    
    # Convolutional Layer 1: trích chọn đặc trưng từ hình ảnh
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Convolutional Layer 2: trích chọn thêm các đặc trưng phức tạp hơn
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Fully Connected Layer: sử dụng đặc trưng đã trích chọn để dự đoán
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


In [11]:
input_shape = (28, 28, 1)  # Kích thước ảnh
num_classes = 26  # Số lượng lớp (chữ cái từ A-Z)

# Xây dựng mô hình CNN
cnn_model = build_cnn_model(input_shape, num_classes)

# In cấu trúc mô hình
cnn_model.summary()
cnn_model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

D:\New folder\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │         3,354 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 227,098 (887.10 KB)

 Trainable params: 227,098 (887.10 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 202s 40ms/step - accuracy: 0.8285 - loss: 0.5966 - val_accuracy: 0.9780 - val_loss: 0.0790
Epoch 2/10
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 183s 36ms/step - accuracy: 0.9533 - loss: 0.1629 - val_accuracy: 0.9830 - val_loss: 0.0613
Epoch 3/10
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 131s 28ms/step - accuracy: 0.9647 - loss: 0.1251 - val_accuracy: 0.9863 - val_loss: 0.0509
Epoch 4/10
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 164s 35ms/step - accuracy: 0.9697 - loss: 0.1069 - val_accuracy: 0.9873 - val_loss: 0.0473
Epoch 5/10
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 154s 33ms/step - accuracy: 0.9719 - loss: 0.0982 - val_accuracy: 0.9881 - val_loss: 0.0438
Epoch 6/10
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 260s 56ms/step - accuracy: 0.9742 - loss: 0.0910 - val_accuracy: 0.9884 - val_loss: 0.0420
Epoch 7/10
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 295s 63ms/step - accuracy: 0.9758 - loss: 0.0861 - val_accuracy: 0.9893 - val_loss: 0.0386
Epoch 8/10
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 300s 64ms/step - accuracy: 

In [13]:
#Đánh giá mô hình
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

2328/2328 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - accuracy: 0.9897 - loss: 0.0369
Loss: 0.03586139157414436, Accuracy: 0.9904550909996033


In [15]:
from tensorflow.keras.saving import save_model
save_model(cnn_model, 'cnn_model.keras')
from tensorflow.keras.models import load_model
#Đường dẫn đến file mô hình đã lưu
model_path = 'cnn_model.keras'
#Load mô hình từ file đã lưu
loaded_model = load_model(model_path)
#In thông tin cấu trúc của mô hình
loaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │         3,354 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 681,296 (2.60 MB)

 Trainable params: 227,098 (887.10 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 454,198 (1.73 MB)

In [16]:
from sklearn.metrics import classification_report, confusion_matrix

In [19]:
# Đánh giá mô hình
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print(f"Loss trên tập kiểm tra: {loss}")
print(f"Độ chính xác trên tập kiểm tra: {accuracy}")

# Dự đoán trên tập kiểm tra
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Báo cáo đánh giá
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred_classes, target_names=[chr(i + ord('A')) for i in range(26)]))

# Ma trận nhầm lẫn
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_true, y_pred_classes))

2328/2328 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - accuracy: 0.9897 - loss: 0.0369
Loss trên tập kiểm tra: 0.03586139157414436
Độ chính xác trên tập kiểm tra: 0.9904550909996033
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 26s 11ms/step

Classification Report:

              precision    recall  f1-score   support

           A       0.99      1.00      0.99      2806
           B       0.99      0.99      0.99      1673
           C       0.99      0.99      0.99      4742
           D       0.97      0.96      0.97      2044
           E       0.99      0.99      0.99      2214
           F       0.99      0.97      0.98       231
           G       0.99      0.97      0.98      1183
           H       0.98      0.98      0.98      1466
           I       1.00      0.99      0.99       237
           J       0.98      0.98      0.98      1668
           K       0.98      0.99      0.98      1132
           L       0.98      0.99      0.99      2319
           M       0.99      0.99      0.99      2487